In [ ]:
from bs4 import BeautifulSoup
import requests
import random

# note for weight value should be 10-120
# checkValueValidation(10, 120, this)

# note for tall value should be 10-120
# checkValueValidation(100, 200, this)

def get_important_information_to_fill_weight_tall():
    pass

def scrape_FitnessWeightHeightMesaures_page(cookies):
    response = requests.get(
        'https://jolearn.jo/index.php?r=EFitness/FitnessWeightHeightMesaures/create',
        cookies=cookies,
    )
    return response.text

def get_grades_data(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')

    # Find the select dropdown by ID
    select_dropdown = soup.find('select', {'id': 'FitnessSchoolSort_ViewCodeNameWHMeasures_GradeID'})

    # Check if the select dropdown is found
    if select_dropdown:
        # Extract options
        grades = []
        for option in select_dropdown.find_all('option'):
            value = option.get('value')
            text = option.get_text(strip=True)
            if "ختر" not in text :
                grades.append({'value': value, 'text': text})
        # print(grades)
        return grades
    
def get_schools_data(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')

    # Find the select dropdown by ID
    select_dropdown = soup.find('select', {'id': 'SchoolID'})

    # Check if the select dropdown is found
    if select_dropdown:
        # Extract options
        schools = []
        for option in select_dropdown.find_all('option'):
            value = option.get('value')
            text = option.get_text(strip=True)
            if len(value):
                schools.append({'value': value, 'text': text})
        # print(schools)
        return schools
    
def scrape_students_FitnessWeightHeightMesaures_html(cookies , schoolId , gradeId):
    data = {
    'viewCodeName': 'ViewCodeNameWHMeasures',
    'SchoolID': schoolId,
    'GradeID': gradeId,
    # 'SchoolID': '4B9EEE39-FC46-4AA7-A33D-1F3FE5E99AC1',
    # 'GradeID': '6363D458-760B-4E5C-8C1D-E15DDA9C13A1',
    }

    response = requests.post(
        'https://jolearn.jo/index.php?r=EFitness/FitnessWeightHeightMesaures/GetListOfAllStudentsPeriods',
        cookies=cookies,
        data=data,
    )
    return response.text

def get_students_FitnessWeightHeightMesaures_data(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')

    # Find all rows with class 'weightHeightMesauresRow'
    rows = soup.find_all('tr', class_='weightHeightMesauresRow')

    # Extract names and corresponding MesaureStudentID values and store in a list of dictionaries
    data = []
    for row in rows:
        name = row.find('td').get_text(strip=True)
        mesaure_student_id = row.find('input', {'name': 'MesaureStudentID'})['value']
        test_period_id = row.find('input', {'name': 'testPeriodID'})['value']
        data.append({'name': name, 'mesaure_student_id': mesaure_student_id, 'test_period_id': test_period_id})

    return data

def random_request_payload_FitnessWeightHeightMesaures(students_data,w_range=(85, 100) , h_range=(100, 120) ):
    # Define the weight and height ranges
    weight_range = w_range
    height_range = h_range

    # Generate the dictionary
    fitness_data = []
    for item in students_data:
        entry = {
            "StudentID": item['mesaure_student_id'],
            "Weight": str(random.randint(*weight_range)),
            "Height": str(random.randint(*height_range)),
            "TestPeriod": item['test_period_id']
        }
        fitness_data.append(entry)

    # Convert the list of dictionaries to a JSON-like string
    fitness_data_str = {'FitnessWeightHeightMesaures':str(fitness_data).replace("'", '"')}
    return fitness_data_str

# suggestions to make the code better :
# 1. scrape all students data at once then posting them all together
# 2. login to microsoft and get the cookies with requests library instead of selenium library

# cookies = {
#     'PHPSESSID': 'r86mi4ope2snec78rb8h7emk6d',
#     'ARRAffinity': 'afb7226a31ec5e26360c36e5e18645c7826aa5dd89a26f31b33c5ca2e1c5d7bd',
#     'ARRAffinitySameSite': 'afb7226a31ec5e26360c36e5e18645c7826aa5dd89a26f31b33c5ca2e1c5d7bd'
# }

# FitnessWeightHeightMesaures_page = scrape_FitnessWeightHeightMesaures_page(cookies)
# schools = [ i['value'] for i in get_schools_data(FitnessWeightHeightMesaures_page)][0]
# grades = get_grades_data(FitnessWeightHeightMesaures_page)

# for grade in grades:
#     html_text = scrape_students_FitnessWeightHeightMesaures_html(cookies , schools , grade['value'])
#     students_data = [i for i in get_students_FitnessWeightHeightMesaures_data(html_text)]
#     request_payload = random_request_payload_FitnessWeightHeightMesaures(students_data)
    
#     response = requests.post(
#         'https://jolearn.jo/index.php?r=EFitness/FitnessWeightHeightMesaures/create',
#         cookies=cookies,
#         data=request_payload
#     )
#     print(" تم تعبئة الصف بنجاح " + grade['text'])
#     print(response.status_code )
    

In [ ]:
controlled_ranodmness = {'سابع' : [(140,150) ,(48,55)] ,
                        'ثامن' : [(148,156) ,(55,62)] ,
                        'تاسع' : [(152,160) ,(60,69)] ,
                        'عاشر' : [(157,164) ,(64,71)] ,
                        'حادي عشر' : [(163,170) ,(64,75)] ,
                        'ثاني عشر' : [(163,175) ,(65,78)]}

for grade in grades:
    for item in controlled_ranodmness:
        if item in grade['text']:
            h_range , w_range = controlled_ranodmness[item]
            request_payload = random_request_payload_FitnessWeightHeightMesaures(students_data ,h_range=h_range,w_range=w_range)
            print(grade)
        else:
            print('not found')
    # grade['text']

In [45]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import wfuzz
from urllib.parse import unquote , quote

def custom_encode(s):
    # Encode only square brackets
    s = s.replace('[', '%5B').replace(']', '%5D')
    return s

def random_request_payload_previousFitnessScore(students_data , school_id ,previousFitnessScore_data , value_ranges=[]):
    # Generate the dictionary
    fitness_data = []
    
    trunkFlexion = previousFitnessScore_data['trunkFlexion_test_data']
    speedAgility = previousFitnessScore_data['speedAgility_test_data']
    flexibility = previousFitnessScore_data['flexibility_test_data']
    running1600m = previousFitnessScore_data['running1600m_test_data']
    inclinedArmFlexion = previousFitnessScore_data['inclinedArmFlexion_test_data']
    
    
    for item in students_data:
        student_id = item['mesaure_student_id']
        
        random_second = random.randint(0, 60)
        random_milliseconds = random.randint(0, 99)
        
        random_value_0 = random.randint(value_ranges[0][0], value_ranges[0][1])
        random_value_1 = f'{random.randint(value_ranges[1][0], value_ranges[1][1])}.{random_milliseconds}'
        random_value_2 = random.randint(value_ranges[2][0], value_ranges[2][1])
        random_value_3 = f'{random.randint(value_ranges[3][0], value_ranges[3][1])}.{random_second}'
        random_value_4 = random.randint(value_ranges[4][0], value_ranges[4][1])
        
        trunkFlexion_entry = f"FitnessScore[Age]=16&FitnessScore[Gender]=1&FitnessScore[StudentId]={student_id}&FitnessScore[SchoolId]={school_id}&FitnessScore[TestMaster]={trunkFlexion['Test_master']}&FitnessScore[TestType]={trunkFlexion['Test_type']}&FitnessScore[TestTypeId]={trunkFlexion['id']}&FitnessScore[Criteria]={random_value_0}"
        speedAgility_entry = f"FitnessScore[Age]=16&FitnessScore[Gender]=1&FitnessScore[StudentId]={student_id}&FitnessScore[SchoolId]={school_id}&FitnessScore[TestMaster]={speedAgility['Test_master']}&FitnessScore[TestType]={speedAgility['Test_type']}&FitnessScore[TestTypeId]={speedAgility['id']}&FitnessScore[Criteria]={random_value_1}"
        flexibility_entry = f"FitnessScore[Age]=16&FitnessScore[Gender]=1&FitnessScore[StudentId]={student_id}&FitnessScore[SchoolId]={school_id}&FitnessScore[TestMaster]={flexibility['Test_master']}&FitnessScore[TestType]={flexibility['Test_type']}&FitnessScore[TestTypeId]={flexibility['id']}&FitnessScore[Criteria]={random_value_2}"
        running1600m_entry = f"FitnessScore[Age]=16&FitnessScore[Gender]=1&FitnessScore[StudentId]={student_id}&FitnessScore[SchoolId]={school_id}&FitnessScore[TestMaster]={running1600m['Test_master']}&FitnessScore[TestType]={running1600m['Test_type']}&FitnessScore[TestTypeId]={running1600m['id']}&FitnessScore[Criteria]={random_value_3}"
        inclinedArmFlexion_entry = f"FitnessScore[Age]=16&FitnessScore[Gender]=1&FitnessScore[StudentId]={student_id}&FitnessScore[SchoolId]={school_id}&FitnessScore[TestMaster]={inclinedArmFlexion['Test_master']}&FitnessScore[TestType]={inclinedArmFlexion['Test_type']}&FitnessScore[TestTypeId]={inclinedArmFlexion['id']}&FitnessScore[Criteria]={random_value_4}"

        fitness_data.extend([custom_encode(trunkFlexion_entry ), custom_encode(speedAgility_entry) , custom_encode(flexibility_entry) , custom_encode(running1600m_entry) , custom_encode(inclinedArmFlexion_entry) ])

    return fitness_data

def enocode_string(string):
    # encode the string
    encoded_string = quote(string)
    return encoded_string

def scrape_previousFitnessScore_data(html):

    # ثتي_الجذع_من_الرقود: trunkFlexion
    # السرعة_والرشاقة: speedAgility
    # مرونة: flexibility
    # 1600م_جري: running1600m
    # ثتي_الذراعين_من_الانبطاح_المائل: inclinedArmFlexion

    soup = BeautifulSoup(html, 'html.parser')

    # Find the first <tr> element with class "fitnessScoreRow"
    first_fitness_row = soup.find('tr', class_='fitnessScoreRow')

    td_elements = first_fitness_row.find_all('td')

    inputs = td_elements[5:10]

    trunkFlexion_test_data = {'Test_master' : inputs[0].find('input')['data--test-master'] , 'Test_type':inputs[0].find('input')['data--test-type'] , 'id':inputs[0].find('input')['data--test-type-id']}
    speedAgility_test_data = {'Test_master' : inputs[1].find('input')['data--test-master'] , 'Test_type':inputs[1].find('input')['data--test-type'] , 'id':inputs[1].find('input')['data--test-type-id']}
    flexibility_test_data = {'Test_master' : inputs[2].find('input')['data--test-master'] , 'Test_type':inputs[2].find('input')['data--test-type'] , 'id':inputs[2].find('input')['data--test-type-id']}
    running1600m_test_data = {'Test_master' : inputs[3].find('input')['data--test-master'] , 'Test_type':inputs[3].find('input')['data--test-type'] , 'id':inputs[3].find('input')['data--test-type-id']}
    inclinedArmFlexion_test_data = {'Test_master' : inputs[4].find('input')['data--test-master'] , 'Test_type':inputs[4].find('input')['data--test-type'] , 'id':inputs[4].find('input')['data--test-type-id']}

    return {
            'trunkFlexion_test_data':trunkFlexion_test_data,
            'speedAgility_test_data':speedAgility_test_data,
            'flexibility_test_data':flexibility_test_data,
            'running1600m_test_data':running1600m_test_data,
            'inclinedArmFlexion_test_data':inclinedArmFlexion_test_data,
            }

def scrape_previousFitnessScore_page (cookies , schoolId , gradeId):

    data = {
        'viewCodeName': 'PreviousFitnessScore',
        'SchoolID': schoolId,
        'GradeID': gradeId,
    }

    response = requests.post(
        'https://jolearn.jo/index.php?r=EFitness/PreviousFitnessScore/GetListOfAllStudentsPeriod',
        cookies=cookies,
        data=data,
    )
    return response.text

def wfuzz_function(url, fuzz_list,cookies,body_postdata,method='POST',proxies = None):
    """دالة استخدمها لارسال طلب بوست بشكل سريع

    Args:
        fuzz_list (list): قائمة في بيانات الطلاب المراد ادخالها
        headers (tuple-list): راسيات الطلب او الركويست
        body_postdata (str): جسم البوست داتا
        method (str, optional): طريقة الطلب. Defaults to 'POST'.

    Returns:
        any : تعود بقائمة الطلبات غير الناجحة
    """    
    unsuccessful_requests=[]
    with tqdm(total=len(fuzz_list), bar_format='{postfix[0]} {n_fmt}/{total_fmt}',
            postfix=["uploaded ", {"value": 0}]) as t:
            s = wfuzz.get_payloads([fuzz_list])
            for idx , r in enumerate(s.fuzz(
                            url=url ,
                            # hc=[404] , 
                            # payloads=[("list",fuzz_list)] ,
                            cookie=cookies ,
                            postdata = body_postdata ,
                            proxies= proxies ,
                            method= method
                            ),start =1):
                    
                t.postfix[1]["value"] = idx
                t.update()    
            #     print(r)
            #     print(r.content)
            #     print(r.history.code) # كود الركويست
                if r.history.code != 200 :
                    unsuccessful_requests.append(r.description)
    return unsuccessful_requests

cookies = {
    'PHPSESSID' : '62p6dnpuvrpusigntniv21nrmd', 
    'ARRAffinity' : 'afb7226a31ec5e26360c36e5e18645c7826aa5dd89a26f31b33c5ca2e1c5d7bd', 
    'ARRAffinitySameSite' : 'afb7226a31ec5e26360c36e5e18645c7826aa5dd89a26f31b33c5ca2e1c5d7d'
}

previousFitnessScore_data = ''
FitnessWeightHeightMesaures_page = scrape_FitnessWeightHeightMesaures_page(cookies)
schools = [ i['value'] for i in get_schools_data(FitnessWeightHeightMesaures_page)][0]
grades = get_grades_data(FitnessWeightHeightMesaures_page)
url = 'https://jolearn.jo/index.php?r=EFitness/PreviousFitnessScore/create'

proxies = [('127.0.0.1','8080','HTTP')]

controlled_ranodmness_previous_fitness_score = {'سابع' : [(20,27) ,(10,11),(24,29),(7,8),(17,26)] ,
                        'ثامن' : [(20,29) ,(10,11),(25,28),(6,7),(19,25)] ,
                        'تاسع' : [(25,30) ,(10,11),(27,31),(6,7),(25,27)] ,
                        'عاشر' : [(27,35) ,(10,10),(28,32),(6,7),(24,29)] ,
                        'حادي عشر' : [(30,36) ,(10,11),(30,35),(6,7),(25,32)] ,
                        'ثاني عشر' : [(34,40) ,(10,11),(32,36),(6,7),(27,35)]}


for grade in grades:
    html_text = scrape_students_FitnessWeightHeightMesaures_html(cookies , schools , grade['value'])
    if not len(previousFitnessScore_data) :
        response = scrape_previousFitnessScore_page(cookies , schools ,grade['value'])
        previousFitnessScore_data = scrape_previousFitnessScore_data(response)
        
    students_data = [i for i in get_students_FitnessWeightHeightMesaures_data(html_text)]
    # request_payload = random_request_payload_previousFitnessScore(students_data , previousFitnessScore_data)
    
    for item in controlled_ranodmness_previous_fitness_score:
        if item in grade['text']:
            value_ranges = controlled_ranodmness_previous_fitness_score[item]
            request_payload = random_request_payload_previousFitnessScore(students_data , schools , previousFitnessScore_data , value_ranges)
            # [f'{i[0]}={i[1]}' for i in cookies.items()]
            print(grade)
        # else:
        #     print('not found')


    wfuzz_cookie = [f"{i[0]}={i[1]}" for i in cookies.items()]
    
    unsuccessful_requests = wfuzz_function(url , request_payload , wfuzz_cookie ,'FUZZ' ,method='POST',proxies = proxies)

    while len(unsuccessful_requests) != 0:
        unsuccessful_requests = wfuzz_function(url , unsuccessful_requests , wfuzz_cookie ,'FUZZ' ,method='POST',proxies = proxies)

    print(" تم تعبئة الصف بنجاح " + grade['text'])


{'value': '6363D458-760B-4E5C-8C1D-E15DDA9C13A1', 'text': 'الصف السابع'}


uploaded  410/410


 تم تعبئة الصف بنجاح الصف السابع
{'value': '78E1D9DB-5E83-4461-BDD7-BDD0733B1C5C', 'text': 'الصف الثامن'}


uploaded  490/490


 تم تعبئة الصف بنجاح الصف الثامن
{'value': '42B27C7A-37D1-41AD-B8E3-4E1C999EC14A', 'text': 'الصف التاسع'}


...

 تم تعبئة الصف بنجاح الصف الحادي عشر العلمي
{'value': '534CCEEE-9BB4-4C13-BB63-D04FDAC930C7', 'text': 'الصف الثاني عشر الأدبي'}


uploaded  285/285


 تم تعبئة الصف بنجاح الصف الثاني عشر الأدبي
{'value': '4DF525CD-7420-4D81-AFC9-3D1784B8D8E4', 'text': 'الصف الثاني عشر العلمي'}


uploaded  70/70

 تم تعبئة الصف بنجاح الصف الثاني عشر العلمي
